In [1]:
import pandas as pd
import numpy as np
uof1 = pd.read_excel('qwe/USE1.xlsx')
uof2 = pd.read_excel('qwe/USE2.xlsx')
uof = pd.concat([uof1, uof2], ignore_index=True)

In [2]:
uof = uof[(uof['IncidentDate'] >= '2024-01-01') & (uof['IncidentDate'] < '2024-04-01')]
uof.head()

,IncidentDate,IncidentTime,Incident Location: Street/Highway,Incident Location: Public Transport,Incident Location: Retail Premises,"Incident Location: Open ground (e.g. park, car park, field)",Incident Location: Licensed Premises,Incident Location: Sports or Event Stadia,Incident Location: Hospital/A&E (non-mental-health setting),Incident Location: Mental Health Setting,...,SubjectNatureOfInjury,SubjectMedOffered,SubjectMedProvided,Outcome: Made off/escaped,Outcome: Arrested,Outcome: Hospitalised,Outcome: Detained - Mental Health Act,Outcome: Fatality,Outcome: Other,Outcome: No Further Action
96251,2024-01-01,23:00:00,No,No,No,No,No,No,No,No,...,NaN,NaN,NaN,No,Yes,No,No,No,No,No
96252,2024-01-01,01:30:00,No,No,No,Yes,No,No,No,No,...,NaN,NaN,NaN,No,No,No,No,No,No,Yes
96253,2024-01-01,16:18:00,Yes,No,No,No,No,No,No,No,...,NaN,NaN,NaN,No,Yes,No,No,No,No,No
96254,2024-01-01,23:02:00,No,No,No,No,No,No,No,No,...,Minor,Yes,Yes,No,Yes,Yes,No,No,No,No
96255,2024-01-01,17:13:00,Yes,No,No,No,No,No,No,No,...,NaN,NaN,NaN,No,Yes,No,No,No,No,No


In [3]:
check = ['SubjectAge']
base = ['Borough']
outcome = [col for col in uof.columns if col.startswith('Outcome:')]
training = []

temp = uof[check+base+outcome]
grouped = temp.groupby(['Borough'])
total_counts = grouped.size()

underaged_counts = temp[(temp['SubjectAge'] == "0-10")|(temp['SubjectAge'] == "11-17")].groupby(['Borough']).size()
percent_underaged = (underaged_counts).reindex(grouped.indices, fill_value=0)
temp['Under18'] = temp.set_index(['Borough']).index.map(percent_underaged)
training.append('Under18')

underaged_counts = temp[(temp['SubjectAge'] == "18-34")].groupby(['Borough']).size()
percent_underaged = (underaged_counts).reindex(grouped.indices, fill_value=0)
temp['Age18to34'] = temp.set_index(['Borough']).index.map(percent_underaged)
training.append('Age18to34')

underaged_counts = temp[(temp['SubjectAge'] == "35-49")].groupby(['Borough']).size()
percent_underaged = (underaged_counts).reindex(grouped.indices, fill_value=0)
temp['Age35to49'] = temp.set_index(['Borough']).index.map(percent_underaged)
training.append('Age35to49')

over50_counts = temp[(temp['SubjectAge'] == "50-64")|(temp['SubjectAge'] == "65 and over")].groupby(['Borough']).size()
percent_over50 = (over50_counts).reindex(grouped.indices, fill_value=0)
temp['Over50'] = temp.set_index(['Borough']).index.map(percent_over50)
training.append('Over50')

over50_counts = temp[(temp['SubjectAge'] == "50-64")|(temp['SubjectAge'] == "65 and over")].groupby(['Borough']).size()
percent_over50 = (over50_counts).reindex(grouped.indices, fill_value=0)
temp['Over50'] = temp.set_index(['Borough']).index.map(percent_over50)
training.append('Over50')

firearmsaimed_counts = temp[(temp['Outcome: Arrested'] == "Yes")].groupby(['Borough']).size()
percent_firearmsaimed = (firearmsaimed_counts).reindex(grouped.indices, fill_value=0)
temp['Arrested'] = temp.set_index(['Borough']).index.map(percent_firearmsaimed)
training.append('Arrested')

firearmsaimed_counts = temp[(temp['Outcome: Hospitalised'] == "Yes")].groupby(['Borough']).size()
percent_firearmsaimed = (firearmsaimed_counts).reindex(grouped.indices, fill_value=0)
temp['Hospitalised'] = temp.set_index(['Borough']).index.map(percent_firearmsaimed)
training.append('Hospitalised')

firearmsaimed_counts = temp[(temp['Outcome: Detained - Mental Health Act'] == "Yes")].groupby(['Borough']).size()
percent_firearmsaimed = (firearmsaimed_counts).reindex(grouped.indices, fill_value=0)
temp['DetainedMHA'] = temp.set_index(['Borough']).index.map(percent_firearmsaimed)
training.append('DetainedMHA')

firearmsaimed_counts = temp[(temp['Outcome: Other'] == "Yes")].groupby(['Borough']).size()
percent_firearmsaimed = (firearmsaimed_counts).reindex(grouped.indices, fill_value=0)
temp['Other'] = temp.set_index(['Borough']).index.map(percent_firearmsaimed)
training.append('Other')

temp['UoF Cases'] = temp.groupby(['Borough'])['Borough'].transform('count')
temp['Log of UoF Cases'] = np.log(temp['UoF Cases'])
training.append('Log of UoF Cases')

temp = temp.drop(columns=(check+outcome))
temp = temp.drop(columns=['UoF Cases'])
temp = temp.fillna(0)
temp.drop_duplicates(inplace=True)
temp = temp.reset_index()
temp = temp.drop(columns=['index'])
temp.head(50)

C:\Users\Wei Xun\AppData\Local\Temp\ipykernel_16936\3867687808.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['Under18'] = temp.set_index(['Borough']).index.map(percent_underaged)
C:\Users\Wei Xun\AppData\Local\Temp\ipykernel_16936\3867687808.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['Age18to34'] = temp.set_index(['Borough']).index.map(percent_underaged)
C:\Users\Wei Xun\AppData\Local\Temp\ipykernel_16936\3867687808.py:22: SettingWithCopyWarning: 
A value is trying to be set on a 

,Borough,Under18,Age18to34,Age35to49,Over50,Arrested,Hospitalised,DetainedMHA,Other,Log of UoF Cases
0,Hillingdon,43,330,196,61,419,52,49,69,6.445720
1,Tower Hamlets,94,609,362,59,757,54,63,111,7.024649
2,Lewisham,127,544,358,125,763,80,60,151,7.050989
3,Newham,180,516,290,113,801,46,26,89,7.002156
4,Southwark,165,580,247,77,676,63,65,96,6.974479
5,Lambeth,127,548,278,97,661,49,67,126,6.956545
6,Brent,84,443,247,73,555,51,54,80,6.741701
7,Hounslow,131,473,278,88,581,86,110,80,6.877296
8,City of Westminster,158,1502,566,170,1540,139,116,253,7.781556
9,Islington,64,456,274,77,558,48,58,111,6.769642


In [4]:
temp.to_csv('data/output_files/predict.csv', index=False)